# Benchmark of intersections of two lists

In [3]:
from functools import total_ordering
@total_ordering
class Element:
    def __init__(self, val: int):
        self.val = val

    def __eq__(self, o: object) -> bool:
        if not isinstance(o, Element):
            return NotImplemented
        return self.val == o.val

    def __lt__(self, o: object) -> bool:
        if not isinstance(o, Element):
            return NotImplemented
        return self.val < o.val

    def __hash__(self) -> int:
        return self.val

import time
def time_fct(l1, l2, fct):
    start = time.time()
    fct(l1, l2)
    return time.time() - start

import random
def run(size, fct):
    l1 = [Element(random.randint(0, size)) for i in range(size)]
    l2 = [Element(random.randint(0, size)) for i in range(size)]
    return time_fct(l1, l2, fct)

def test(fct):
    size = 1000
    l1 = [Element(random.randint(0, size)) for i in range(size)]
    l2 = [Element(random.randint(0, size)) for i in range(size)]
    inter = fct(l1, l2)
    assert all((not e1 in l2) or (e1 in inter) for e1 in l1)
    assert all((not e2 in l1) or (e2 in inter) for e2 in l2)

## Python3 implementation

In [4]:
def py_intersection_1(l1, l2):
    return list(set(l1) & set(l2))

def py_intersection_2(l1, l2):
    return list(filter(lambda x: x in l1, l2))

def py_intersection_3(l1, l2):
    return [x for x in l1 if x in l2]

def py_intersection_4(l1, l2):
    s = set(l1)
    return [x for x in l2 if x in s]

def py_intersection_5(l1, l2):
    d = { x:1 for x in l1 }
    return [x for x in l2 if x in d]

def py_intersection_6(l1, l2):
    l1.sort()
    l2.sort()
    res = []
    i1, i2 = 0, 0
    while i1 < len(l1) and i2 < len(l2):
        if l1[i1] < l2[i2]:
            i1 += 1
        elif l1[i1] > l2[i2]:
            i2 += 1
        else:
            res.append(l1[i1])
            i1 += 1
            i2 += 1
    return res

intersections_py = [py_intersection_1, py_intersection_2, py_intersection_3, py_intersection_4, py_intersection_5, py_intersection_6]

## C++ implementation

In [5]:
import my_set

def cpp_intersection_1(l1, l2):
    return my_set.intersection_1(l1, l2)

def cpp_intersection_2(l1, l2):
    return my_set.intersection_2(l1, l2)

intersections_cpp = [cpp_intersection_1, cpp_intersection_2]

## Tests

In [6]:
intersections_fct = intersections_cpp + intersections_py
for fct in intersections_fct:
    try:
        test(fct)
        print("%s: Succeed" % fct.__name__)
    except AssertionError:
        print("%s: Failure" % fct.__name__)

cpp_intersection_1: Succeed
cpp_intersection_2: Succeed
py_intersection_1: Succeed
py_intersection_2: Succeed
py_intersection_3: Succeed
py_intersection_4: Succeed
py_intersection_5: Succeed
py_intersection_6: Succeed


## Benchmark

In [4]:
sizes = [10, 100, 1000, 10000, 100000, 1000000, 10000000, 100000000]
time_threshold = 1 # seconds

In [5]:
intersections_fct = intersections_cpp + intersections_py
for size in sizes:
    print("\nSize:", size)
    scores = [(fct, run(size, fct)) for fct in intersections_fct]
    scores.sort(key=lambda x: x[1])
    best_score = scores[0][1]
    for score in scores:
        print("\t%s: %.3f sec" % (score[0].__name__, score[1]))
        if score[1] - best_score > time_threshold:
            intersections_fct.remove(score[0])
            print("\t\tremove %s" % scores[-1][0].__name__)


Size: 10
	py_intersection_5: 0.000 sec
	cpp_intersection_1: 0.000 sec
	py_intersection_4: 0.000 sec
	py_intersection_1: 0.000 sec
	py_intersection_3: 0.000 sec
	py_intersection_2: 0.000 sec
	py_intersection_6: 0.000 sec
	cpp_intersection_2: 0.000 sec

Size: 100
	cpp_intersection_1: 0.000 sec
	py_intersection_4: 0.000 sec
	py_intersection_5: 0.000 sec
	py_intersection_1: 0.000 sec
	cpp_intersection_2: 0.000 sec
	py_intersection_6: 0.000 sec
	py_intersection_2: 0.001 sec
	py_intersection_3: 0.001 sec

Size: 1000
	py_intersection_5: 0.000 sec
	py_intersection_4: 0.000 sec
	cpp_intersection_1: 0.001 sec
	py_intersection_1: 0.001 sec
	py_intersection_6: 0.004 sec
	cpp_intersection_2: 0.005 sec
	py_intersection_3: 0.095 sec
	py_intersection_2: 0.118 sec

Size: 10000
	cpp_intersection_1: 0.003 sec
	py_intersection_1: 0.006 sec
	py_intersection_4: 0.007 sec
	py_intersection_5: 0.008 sec
	cpp_intersection_2: 0.058 sec
	py_intersection_6: 0.066 sec
	py_intersection_3: 10.367 sec
		remove py_int